In [3]:
# 필요한 패키지 설치
import requests
from bs4 import BeautifulSoup
import io
import zipfile
import xmltodict
from io import BytesIO
from zipfile import ZipFile
from xml.etree.ElementTree import parse
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

## 고유번호 개발 가이드
1. 요청 파라미터
- 고유번호 corp_code 
- 정식명칭 corp_name
- 종목코드 stock_code
- 최종변경일자 modify_date

2. 추출 결과값
- 

In [4]:
# https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018
KEY = '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85'

url = '	https://opendart.fss.or.kr/api/corpCode.xml'
params = {'crtfc_key' : KEY}

response = requests.get(url, params = params).content


# 압축 파일 풀어서 xml파일 디렉토리에 저장하기
with ZipFile(BytesIO(response)) as zipfile:
    zipfile.extractall('corpCode')

#corpcode xml 파일 생김

In [5]:
# xml 파일 읽어오기, 여기서 고유번호와 종목명이 포함된 리스트를 만들 것임
xmlTree = parse('./corpCode/CORPCODE.xml')
root = xmlTree.getroot()
raw_list = root.findall('list')

corp_list = []

for l in range(len(raw_list)):
    corp_code = raw_list[l].findtext('corp_code')
    corp_name = raw_list[l].findtext('corp_name')
    stock_code = raw_list[l].findtext('stock_code')
    modify_date = raw_list[l].findtext('modify_date')
    
    corp_list.append([corp_code, corp_name, stock_code, modify_date])
    
#corp_list 형성

In [6]:
corp_list

[['00434003', '다코', ' ', '20170630'],
 ['00434456', '일산약품', ' ', '20170630'],
 ['00430964', '굿앤엘에스', ' ', '20170630'],
 ['00432403', '한라판지', ' ', '20170630'],
 ['00388953', '크레디피아제이십오차유동화전문회사', ' ', '20170630'],
 ['00179984', '연방건설산업', ' ', '20170630'],
 ['00420143', '브룩스피알아이오토메이션잉크', ' ', '20170630'],
 ['00401111', '매경아이비아이', ' ', '20170630'],
 ['00435534', '캐드뱅크', ' ', '20170630'],
 ['00430186', '엠와이오피삼차유동화전문유한회사', ' ', '20170630'],
 ['00430201', '엠와이오피이차유동화전문유한회사', ' ', '20170630'],
 ['00430210', '엠와이오피일차유동화전문유한회사', ' ', '20170630'],
 ['00430229', '포스미디어', ' ', '20170630'],
 ['00140432', '축복할렐루야', ' ', '20170630'],
 ['00426208', '한국전자화학', ' ', '20170630'],
 ['00433262', 'ConnachtCapitalMarketInvestmentLtd.', ' ', '20170630'],
 ['00433749', '선진아이티', ' ', '20170630'],
 ['00433785', '팀스코리아', ' ', '20170630'],
 ['00196079', '에넥스하이테크', ' ', '20170630'],
 ['00435048', '세이스텝바이스텝혼합형펀드', ' ', '20170630'],
 ['00435057', '유리알파헷지채권혼합형펀드', ' ', '20170630'],
 ['00108843', '한기술정보통신', ' ', '2017063

In [12]:
#corp_list -> corp_df


from pandas import DataFrame
from datetime import datetime

#위에서 생성한 corp_list에 라벨링하여 데이터프레임화

corp_df = DataFrame(corp_list, columns=[
    '고유번호', 
    '정식명칭', 
    '종목코드', 
    '최종변경일자'
])

#corp_df 의 결측치 drop시킨 stock_df (종목코드가 없는 종목을 drop하고, 최종변경일자는 쓰지 않으므로 칼럼 전체를 drop)

stock_df = corp_df[corp_df['종목코드'] != " "]
stock_df = stock_df[['고유번호', '정식명칭', '종목코드']].dropna()
stock_df

,고유번호,정식명칭,종목코드
2008,00260985,한빛네트,036720
2020,00264529,엔플렉스,040130
2021,00358545,동서정보기술,055000
2783,00231567,애드모바일,032600
3886,00247939,씨모스,037600
...,...,...,...
101145,00296078,APS,054620
101146,01118643,로보쓰리에이아이앤로보틱스,238500
101147,00624518,다산솔루에타,154040
101148,01065679,알비케이그룹,215790


In [39]:
# stock_df에서 고유번호를 호출해오는 함수 형성
def goyu_searcher(COMPANYNAME):
    temp_df = stock_df.loc[(stock_df['정식명칭'] == COMPANYNAME)]
    return temp_df.iloc[0, 0]

In [40]:
# stock_df에서 종목 이름을 호출해오는 함수 형성
def corpname_searcher(CORP_CODE):
    temp_df = stock_df.loc[(stock_df['고유번호'] == CORP_CODE)]
    return temp_df.iloc[0, 1]

In [68]:
# 가져온 corp_code를 바탕으로 

def get_items(KEY, CORP_CODE, YEAR, RPT_CODE):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from lxml import html
    from urllib.request import Request, urlopen
    from urllib.parse import urlencode, quote_plus, unquote
    
    
    url = 'https://opendart.fss.or.kr/api/fnlttMultiAcnt.json'
    params={'crtfc_key' : KEY,
            'corp_code' : CORP_CODE, 
            'bsns_year' : YEAR, 
            'reprt_code' : RPT_CODE}
    
    response = requests.get(url, params = params)
    
    import json
    try:
            json_obj = json.loads(response.text)["list"][0]
    except:
        #     json_obj = ["## ERROR", corpname_searcher(CORP_CODE), YEAR, RPT_CODE]
            json_obj = ["## ERROR", corpname_searcher(CORP_CODE), YEAR] #ok
            # json_obj = [corpname_searcher(CORP_CODE), RPT_CODE] ok
        #     json_obj = [corpname_searcher(CORP_CODE), YEAR, RPT_CODE]
        
    
    return json_obj

In [ ]:
get_items(KEY, '00126380', '2018', '11011')

In [69]:
#get_items()를 사용하여 구하고자 하는 기업들의 재무제표 정보 가져오기

corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
year_list = range(2018, 2024)
reprtlist = ['11013', '11012', '11014', '11011']

result_list = []

for corp in corp_list:
    for year in year_list:
        for reprt in reprtlist:
            result_list.append(get_items(KEY, goyu_searcher(corp), year, reprt))
            

In [89]:
# 구한 결과값을 df로 변환


#틀을 먼저 만들고

corp_name_dfcol = []
corp_code_dfcol = []
bsns_year_dfcol = []
reprt_code_dfcol = []

result_list = [dict_row for dict_row in result_list if isinstance(dict_row, dict)]

for dict_row in result_list:
    corp_name_dfcol.append(corpname_searcher(dict_row['corp_code']))
    corp_code_dfcol.append(dict_row['corp_code'])
    bsns_year_dfcol.append(dict_row['bsns_year'])
    reprt_code_dfcol.append(dict_row['reprt_code'])

data = {'기업명': corp_name_dfcol, '종목코드': corp_code_dfcol, '사업년도': bsns_year_dfcol, '분기코드': reprt_code_dfcol}

result_df = pd.DataFrame(data = data)
result_df

,기업명,종목코드,사업년도,분기코드
0,삼성전자,00126380,2018,11013
1,삼성전자,00126380,2018,11012
2,삼성전자,00126380,2018,11014
3,삼성전자,00126380,2018,11011
4,삼성전자,00126380,2019,11013
...,...,...,...,...
79,POSCO홀딩스,00155319,2022,11013
80,POSCO홀딩스,00155319,2022,11012
81,POSCO홀딩스,00155319,2022,11014
82,POSCO홀딩스,00155319,2022,11011


In [ ]:
# 지표 데이터를 넣을 컬럼 리스트 생성

data_dfcol = []

for dict_row in result_list:
    data_dfcol.append(dict_row['account_nm'])


result
# 각각의 지표 데이터를 데이터프레임에 추가하는 부분부터 코딩해야 함

## 하단의 셀은 무시

## OpenDartReader 클래스 사용

In [ ]:
import OpenDartReader
import pandas as pd

dart = OpenDartReader(KEY)
# dfdf = dart.finstate('005930, 032830, 005380, 068270, 005490', 2022)


corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
year_list = range(2018, 2024)
reprtlist = ['11013', '11012', '11014', '11011']


dfs = []
for corp in corp_list:
    for year in year_list:
        for report in reprtlist:
            singledf = dart.finstate(corp, year, reprt_code = report)
            dfs.append(singledf)



result = pd.concat(dfs, ignore_index=True)
result




In [ ]:
result = result.drop(columns=['ord', 'rcept_no', 'stock_code'])
result

In [89]:
dfdfdf.to_csv('./apiresult.csv', encoding='euc-kr')

In [38]:
KEY = '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85'
YEAR = '2021'
RPT_CODE = '11011' 
# 1분기보고서 : 11013
# 반기보고서 : 11012
# 3분기보고서 : 11014
# 사업보고서 : 11011

# 가져오고자 하는 기업의 이름이 담긴 리스트가 있을 거야
corp_list = ['삼성전자', '삼성생명', '현대자동차', '셀트리온', 'POSCO홀딩스']
year_list = range(2018, 2024)

item_list =[
    'bsns_year',
    'stock_code',
    'reprt_code',
    'fs_div',
    'sj_div',
    'account_nm',
    'thstrm_nm',
    'thstrm_dt',
    'thstrm_amount',
    'thstrm_add_amount',
    'frmtrm_nm',
    'frmtrm_dt',
    'frmtrm_amount',
    'frmtrm_add_amount',
    'bfefrmtrm_nm',
    'bfefrmtrm_dt',
    'bfefrmtrm_amount',
    'currency'
]

fs_list = []
for corp in corp_list:
    CORP_CODE = goyu_searcher(corp)
    
    items = get_items(KEY, CORP_CODE, YEAR, RPT_CODE)
    
    for i in range(0, len(items)):
        fs_item_list = []
        for item in item_list:
            try:
                value = items[i].find(item).text
            except:
                value = ''
            fs_item_list.append(value)
            
        fs_list.append(fs_item_list)
        
    print(CORP_CODE)


005930
032830
005380
068270
005490


## dart-fss

In [37]:
!pip install dart-fss

     |████████████████████████████████| 141 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 11.1 MB/s eta 0:00:01
  Using cached fake_useragent-1.1.3-py3-none-any.whl (50 kB)
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
     |████████████████████████████████| 41 kB 3.2 MB/s eta 0:00:01
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11260 sha256=7e1539c7dda11c144fa3a78694581a10de3c2fbada724de8085d932753419184
  Stored in directory: /Users/mj/Library/Caches/pip/wheels/bb/85/47/b7c7338ab52808105f937bd8c04aec5d98a543311ac2c8bed2
Successfully built halo


In [44]:
import dart_fss as dart

KEY = '5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85'
dart.set_api_key(api_key = KEY)

corp_list = dart.get_corp_list()



AttributeError: 'list' object has no attribute 'info'

In [46]:
len(list(corp_list))

101136

In [48]:
# 삼성전자 검색
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0]

# 2012년부터 연간 연결재무제표 불러오기
fs = samsung.extract_fs(bgn_de='20160101')

# 재무제표 검색 결과를 엑셀파일로 저장 ( 기본저장위치: 실행폴더/fsdata )
fs.save()

Annual reports:   0%|          | 0/8 [00:00<?, ?report/s]

'/Users/mj/Desktop/workspace/final_pjt/fsdata/00126380_annual.xlsx'

## 테스트

In [73]:
goyu_searcher('삼성생명')
get_items(KEY, '00126256', '2019', '11011')

['## ERROR', '삼성생명', '2019']

In [10]:
test_response = requests.get("https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85&corp_code=00126380&bsns_year=2018&reprt_code=11011&fs_div=OFS")
test_response.text

200

In [78]:
test_response = requests.get("https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=5b8f0aa6967605a94ba6e9a39b461c2e86cf0d85&corp_code=00126256&bsns_year=2020&reprt_code=11011&fs_div=CFS")
test_response.text

'{"status":"013","message":"조회된 데이타가 없습니다."}'

In [21]:
sum(stock_df['정식명칭'] == '')

0

In [74]:
goyu_searcher('삼성생명')

'00126256'